# Poggi's Conference Acceptance Rate Data Integration

Jupyter Notebook for the processing and integration of the Conference Acceptance Rate Data obtained by Prof. Francesco Poggi, professor at the University of Modena and Reggio Emilia.

*The CORE Conference Ranking provides assessments of major conferences in the computing disciplines.The rankings are managed by the CORE Executive Committee, with periodic rounds for submission of requests for addition or reranking of conferences. Decisions are made by academic committees based on objective data requested as part of the submission process.* (source: CORE)

The data was obtained by web scraping and is provided in JSON format.
____________________________________________________________

For this process, the following CSV files are needed: ```out_citations_and_conferences_location_ready_v2.csv``` and the Conference Acceptance Rate JSON files. 

The first one must be generated running the Notebook ```1 - Citation and Locations Dataset Preparation.ipynb``` that is contained in the ```5 - Conference Locations Ranking Integration``` folder of this Repository.<br>
Poggi's Conference Acceptance Rate JSON files can be downloaded from [here]().

In particular, the following operations are going to be executed:
* Opening of the CSV conference citations and locations dataset
* (Sequential) Reading of the JSON files
* Expansion of the JSON fields
* Processing of the JSON data
* Join of the different processed JSON data
* Join between the distinct conference series name and the Conference Acceptance Rate Data

Lastly, the processed dataset is going to be saved on disk in CSV format

In [223]:
# Libraries Import
import pandas as pd
import numpy as np
import glob

pd.set_option('display.max_columns', None)

## File Paths
Please set your working directory paths.

In [2]:
# ******************* PATHS ********************+

# Dumps Directory Path
path_file_import = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Import/data_acceptance/'

# CSV Exports Directory Path
path_file_export = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/'

## Read and Preparation of the Citation Dataset

In [3]:
df_citations_and_locations = pd.read_csv(path_file_export + 'out_citations_and_conferences_location_ready_v2.csv', low_memory=False, index_col=[0])
print(f'Successfully Imported the Conference Citations and Locations Ready CSV')

Successfully Imported the Conference Citations and Locations Ready CSV


In [4]:
df_citations_and_locations.head(3)

,CitationCount_COCI,CitationCount_Mag,CitationCount_MagEstimated,ConferenceLocation,ConferenceNormalizedName,ConferenceSeriesNormalizedName,Doi,Year
0,10,12,12,"Austin, Texas, United States",disc 2014,disc,10.1007/978-3-662-45174-8_28,2014
1,5,10,10,"Wrocław, Lower Silesian Voivodeship, Poland",esa 2014,esa,10.1007/978-3-662-44777-2_60,2014
2,11,20,20,"Innsbruck, Tyrol, Austria",enter 2013,enter,10.1007/978-3-319-03973-2_13,2013


### Extracion of the Distinct Conference Series from the Conference and Locations Datasets

In [5]:
df_conference_series = df_citations_and_locations.drop_duplicates(subset="ConferenceSeriesNormalizedName")

#filter of the useless columns
df_conference_series = df_conference_series.drop(df_conference_series.columns.difference(["ConferenceSeriesNormalizedName"]), axis=1)

# drop of the nan row
df_conference_series = df_conference_series.dropna(subset={"ConferenceSeriesNormalizedName"})

# reset of the index
df_conference_series = df_conference_series.reset_index(drop=True)

df_conference_series

,ConferenceSeriesNormalizedName
0,disc
1,esa
2,enter
3,dexa
4,icaisc
...,...
5307,infinity
5308,calculemus
5309,agp
5310,sci


## Understanding the Structure of the JSON Files

In [226]:
df_AcmConferences_raw = pd.read_json(path_file_import + 'AcmConferences.json')
print(f'Successfully Imported the AcmConferences JSON')

Successfully Imported the AcmConferences JSON


Print of the dataset structure:

In [15]:
df_AcmConferences_raw.head(3)

,confName,confAcronym,confUrl,info
0,DocEng: Document Engineering,DocEng,https://dl.acm.org/action/doSearch?target=brow...,"[{'year': '19', 'yearUnparsed': 'DocEng '19', ..."
1,"MANPU: Comics Aanalysis, Processing and Unders...",MANPU,https://dl.acm.org/action/doSearch?target=brow...,"[{'year': '16', 'yearUnparsed': 'MANPU '16', '..."
2,Nanoarch: Nanoscale Architectures,Nanoarch,https://dl.acm.org/action/doSearch?target=brow...,"[{'year': '18', 'yearUnparsed': 'NANOARCH '18'..."


### Expansion of the info field
The following stuff was only needed to understand the structure of the dataset

In [79]:
df_first_level = pd.json_normalize(df_AcmConferences_raw.iloc[[0]]['info'])
df_first_level

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,"{'year': '19', 'yearUnparsed': 'DocEng '19', '...","{'year': '17', 'yearUnparsed': 'DocEng '17', '...","{'year': '16', 'yearUnparsed': 'DocEng '16', '...","{'year': '15', 'yearUnparsed': 'DocEng '15', '...","{'year': '14', 'yearUnparsed': 'DocEng '14', '...","{'year': '13', 'yearUnparsed': 'DocEng '13', '...","{'year': '12', 'yearUnparsed': 'DocEng '12', '...","{'year': '11', 'yearUnparsed': 'DocEng '11', '...","{'year': '10', 'yearUnparsed': 'DocEng '10', '...","{'year': '09', 'yearUnparsed': 'DocEng '09', '...","{'year': '08', 'yearUnparsed': 'DocEng '08', '...","{'year': '07', 'yearUnparsed': 'DocEng '07', '...","{'year': '06', 'yearUnparsed': 'DocEng '06', '...","{'year': '05', 'yearUnparsed': 'DocEng '05', '...","{'year': '04', 'yearUnparsed': 'DocEng '04', '...","{'year': '03', 'yearUnparsed': 'DocEng '03', '...","{'year': '02', 'yearUnparsed': 'DocEng '02', '...","{'year': '01', 'yearUnparsed': 'DocEng '01', '..."


In [80]:
pd.json_normalize(df_first_level.iloc[:, 0])

,year,yearUnparsed,yearUrl,submitted,accepted,percAccepted
0,19,DocEng '19,https://dl.acm.org/doi/proceedings/10.1145/334...,77,30,39%


## Processing of the JSON Files

### Definition of the JSON Processing Function

Definition of the processing function (is going to be used for all the different datasets):

In [219]:
def process_json_dataframe(raw_dataframe):
    out_row_list = list()

    for index, row in raw_dataframe.iterrows():

        # expansion from the info field
        df_first_level = pd.json_normalize(row['info'])

        for index_first_level, row_first_level in df_first_level.iterrows():
            # check if there's the year
            try:
                row_first_level['year'] 
            except KeyError:
                continue

            # check if there's the percAccepted
            try:
                row_first_level['percAccepted'] 
            except KeyError:
                continue
            
            # filter of null rows
            if pd.isnull(row_first_level['year']) or pd.isnull(row_first_level['percAccepted']) or pd.isnull(row_first_level['submitted']) or pd.isnull(row_first_level['accepted']):
                continue # skip row

            # filter of some year special cases
            if isinstance(row_first_level['year'], str):
                if row_first_level['year'].__len__() > 4: # special case like "19 Companion", that we need to filter
                    continue # skip row

            # filter of some invalid data
            if "?" in str(row_first_level['submitted']) or "?" in str(row_first_level['accepted']) or "?" in str(row_first_level['percAccepted']):
                continue # skip row

            # normalization of the year format
            year = int(row_first_level['year'])
            if year < 100: # two figures
                year = 2000 + year

            # creation of the support dataframe
            support_dict = dict()
            support_dict['Conf_Acronym'] = row['confAcronym'].lower()
            support_dict['Year'] = year

            # removing points or commas 
            submitted_str = str(row_first_level['submitted']).split(".")[0].split(",")[0].split("+")[0].split(">")[0].split("<")[0].split("(")[0]
            if "~" in submitted_str:
                submitted_str = submitted_str.split("~")[1]
            submitted_str = submitted_str.strip()
            if submitted_str.__len__() == 0 or "--" in submitted_str or not submitted_str.isnumeric():
                continue # skip row
            submitted = int(submitted_str)

            accepted_str = str(row_first_level['accepted']).split(".")[0].split(",")[0].split("+")[0].split(">")[0].split("<")[0].split("(")[0]
            if ":" in accepted_str:
                accepted_str = accepted_str.split(":")[1]
            if "~" in accepted_str:
                accepted_str = accepted_str.split("~")[1]
            accepted_str = accepted_str.strip()
            if accepted_str.__len__() == 0 or "--" in accepted_str or not accepted_str.isnumeric():
                continue # skip row
            accepted = int(accepted_str)
                

            perc_accepted_str = str(row_first_level['percAccepted']).split('%')[0].split("+")[0].split(">")[0].split("<")[0].replace(",", ".")
            if perc_accepted_str.__len__() == 0:
                continue # skip row
            if "~" in perc_accepted_str:
                perc_accepted = float(perc_accepted_str.split("~")[1])
            else:
                perc_accepted = float(perc_accepted_str)

            support_dict['Papers_Submitted'] = submitted
            support_dict['Papers_Accepted'] = accepted
            support_dict['Papers_Perc_Accepted'] = perc_accepted
            
            # putting the values inside our output list
            out_row_list.append(support_dict)

    # conversion of the output list to a dataframe
    return pd.DataFrame(out_row_list) 

### JSON Reading and Processing

In [234]:
output_dataframes_list = list()

poggis_all_jsons = glob.glob(path_file_import + "*.json")

for current_json_name_full in poggis_all_jsons:

    current_json_name = current_json_name_full.split("/")[-1]

    # Open the current CSV
    print(f'Currently processing JSON: {current_json_name}')
    df_current_json = pd.read_json(current_json_name_full)

    # Processing of the json file
    output_dataframes_list.append(process_json_dataframe(df_current_json))

Currently processing JSON: CryptographyConferencesStatistics.json
Currently processing JSON: TheoreticalCSConferencesStatistics.json
Currently processing JSON: NetworkingConferencesStatistics.json
Currently processing JSON: AcmConferences.json
Currently processing JSON: SEConferencesStatistics.json
Currently processing JSON: ComputerSecurityConferencesStatistics.json


Check of the first processed dataset:

In [239]:
if output_dataframes_list.__len__() > 0:
    print(output_dataframes_list[0])

    Conf_Acronym  Year  Papers_Submitted  Papers_Accepted  \
0         crypto  2014               227               60   
1         crypto  2013               227               61   
2         crypto  2012               225               48   
3         crypto  2011               230               42   
4         crypto  2010               202               39   
..           ...   ...               ...              ...   
122          tcc  2010               100               33   
123          tcc  2009               109               33   
124          tcc  2008                81               33   
125          tcc  2007               118               31   
126          tcc  2006                91               31   

     Papers_Perc_Accepted  
0                    26.4  
1                    26.9  
2                    21.3  
3                    18.3  
4                    19.3  
..                    ...  
122                  33.0  
123                  30.3  
124            

## Join Between the Different Processed JSON Files

## Write of the Final CSVs on Disk

Saving the resulting dataframe on disk in CSV format.

In [47]:
# Write of the resulting CSVs on Disk
df_conference_series_with_core_rank.to_csv(path_file_export + 'out_conference_series_with_core_rank.csv')
print(f'Successfully Exported the Joined CSV to {path_file_export}out_conference_series_with_core_rank.csv')

Successfully Exported the Joined CSV to /Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/out_conference_series_with_core_rank.csv


Check of the Exported CSV to be sure that everything went fine.

In [48]:
# Check of the Exported CSV
df_conference_series_with_core_rank = pd.read_csv(path_file_export + 'out_conference_series_with_core_rank.csv', low_memory=False, index_col=[0])
df_conference_series_with_core_rank

,CORE_2008_Rank,CORE_2013_Rank,CORE_2014_Rank,CORE_2017_Rank,CORE_2018_Rank,CORE_2020_Rank,CORE_2021_Rank,ConferenceSeriesNormalizedName,ERA_2010_Rank
0,A,A,A,A,A,A,A,disc,A
1,A,A,A,A,A,A,A,esa,A
2,NaN,C,C,C,C,C,C,enter,C
3,A,B,B,B,B,B,B,dexa,B
4,NaN,C,C,C,C,NaN,NaN,icaisc,C
...,...,...,...,...,...,...,...,...,...
5307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,infinity,NaN
5308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,calculemus,NaN
5309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,agp,NaN
5310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sci,NaN
